<a href="https://colab.research.google.com/github/hugohonda/diario-oficial/blob/master/DataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import re
from pprint import pprint
import itertools as it

In [0]:
try:
  df = pd.read_json('/content/drive/My Drive/TCC/2014-3.json')
except Exception as e:
  print('Could not read input file:')
  print(e)

In [4]:
df = df[df['body'].apply(lambda x: len(x)!=0)]
print('rows with body content: ', len(df))
df.sample(10, random_state=42)

rows with body content:  63807


,body,date,journal,pageNumber,status,url
32043,"Nº 92, sexta-feira, 16 de maio de 2014\n148 IS...",2014-05-16,3,148,True,http://pesquisa.in.gov.br/imprensa/servlet/INP...
33387,"Nº 12, sexta-feira, 17 de janeiro de 2014\n44 ...",2014-01-17,3,44,True,http://pesquisa.in.gov.br/imprensa/servlet/INP...
5297,"Nº 191, sexta-feira, 3 de outubro de 2014\n134...",2014-10-03,3,134,True,http://pesquisa.in.gov.br/imprensa/servlet/INP...
29920,"Nº 72, terça-feira, 15 de abril de 2014 237\nI...",2014-04-15,3,237,True,http://pesquisa.in.gov.br/imprensa/servlet/INP...
3126,"Nº 190, quinta-feira, 2 de outubro de 2014 127...",2014-10-02,3,127,True,http://pesquisa.in.gov.br/imprensa/servlet/INP...
57935,"Nº 231, sexta-feira, 28 de novembro de 2014\n2...",2014-11-28,3,216,True,http://pesquisa.in.gov.br/imprensa/servlet/INP...
8758,"Nº 43, quarta-feira, 5 de março de 2014 107\nI...",2014-03-05,3,107,True,http://pesquisa.in.gov.br/imprensa/servlet/INP...
24476,"Nº 219, quarta-feira, 12 de novembro de 2014 1...",2014-11-12,3,13,True,http://pesquisa.in.gov.br/imprensa/servlet/INP...
35470,"Nº 222, segunda-feira, 17 de novembro de 2014 ...",2014-11-17,3,11,True,http://pesquisa.in.gov.br/imprensa/servlet/INP...
44116,"Nº 75, terça-feira, 22 de abril de 2014 137\nI...",2014-04-22,3,137,True,http://pesquisa.in.gov.br/imprensa/servlet/INP...


In [52]:
sample = df.sample(1)
print('url: ', sample['url'].values[0])
print('index: ', sample.index.values[0])
text = sample['body'].values[0]

def clean_text (text):
  text = re.sub(r'[^\S\n]', ' ', text)
  text = re.sub(r'(?<=[A-Z]\-)\n(?=[A-Z])', '', text)
  text = re.sub(r'^N[\s\S]+?ICP-Brasil.\s\d', '', text)
  return text

def extract_publications (text):
  clean = clean_text(text)
  sentences = re.finditer(r'(?:\n|^)([A-Z]{2,}[A-ZÃÂÁÀẼÊÉÈÍÌÕÔÓÒÚÙÇ\s\d\.,oºª\-\/]{3,})(?:\n)(?![a-z])', clean)
  sentences, sentences_next = it.tee(sentences)
  next(sentences_next)
  publications = []
  for sentence, sentence_next in zip(sentences, sentences_next):
    sent_text = re.sub(r'^\n|\n$', '', clean[sentence.end():sentence_next.start()])
    sent_text = re.sub(r'(?:(?<=E|\-|\,|\:)|(?<=D(?:E|A|O)|N(?:A|O))|(?<=D(?:E|A|O)S|N(?:A|O)S))\n', ' ', sent_text)
    sent_text = re.sub(r'\n(?=(?:E|D(?:E|A|O)S?|N(?:A|O)S?|\-)\s+)', ' ', sent_text)
    sent_text = re.sub(r'[a-z]\-\s|[A-Z]\-[A-Z]', '', sent_text)
    publication = {
        'title': sentence.group(),
        'body': sent_text
    }
    publications.append(publication)
  return publications

pprint(extract_publications(text))

url:  http://pesquisa.in.gov.br/imprensa/servlet/INPDFViewer?jornal=3&pagina=162&data=14/05/2014&totalArquivos=308&captchafield=firstAccess
index:  28161
[{'body': 'Objeto: Aquisição de Mobiliário; HOMOLOGADO à empresa R N\n'
          'MARQUES ARAUJO - EPP (CNPJ 02.717.699/0001-30), Grupo 01, valor '
          'global: R$ 35.944,95 (trinta e cinco mil e novecentos e quarenta\n'
          'e quatro reais e noventa e cinco centavos).',
  'title': '\n'
           'DIRETORIA REGIONAL NO PIAUÍ\n'
           'AVISOS DE HOMOLOGAÇÃO\n'
           'PREGÃO ELETRÔNICO Nº 3/2014\n'},
 {'body': 'Pregoeiro', 'title': '\nLEANDRO CRUZ REZENDE\n'},
 {'body': 'Objeto: Aquisição de Bicicletas; HOMOLOGADO à empresa PAICIPA '
          'EQUIPAMENTOS E SERVIÇOS EIRELI - EPP (CNPJ\n'
          '07.205.127/0001-96), ITEM 01, valor global: R$ 276.190,00 (dzentos '
          'e setenta e seis mil e cento e noventa reais).',
  'title': '\nPREGÃO ELETRÔNICO Nº 4/2014\n'},
 {'body': 'Pregoeira', 'title': '\nFRANC

In [0]:
# print(df['body'][20188])

In [0]:
# locais:
# SECRETARIA
# PREFEITURA MUNICIPAL
# HOSPITAL
# COMANDO MILITAR
# MINISTERIO
# GABINETE
# GERÊNCIA
# CENTRO
# CENTRO REGIONAL / NACIONAL
# INSTITUTO
# COMISSÃO
# CAMPUS
# CONSELHO
# UNIVERSIDADE
# PROCESSO SELETIVO
# MUSEU
# SELEÇÃO
# FUNDO MUNICIPAL
# DIRETORIA
# COMUNICADO

extract_regex = r'extratos?\sd(?:e|o|a)\s(contratos?|termos?\saditivos?|inexigibilidade\sde\slicita.?.?o|parcerias?)'
warning_regex = r'aviso\sde\s(?:(?:(?:licit|anul|retific)(?:a..o))|chamamento\sp.blico)'
# ATA DE REGISTRO DE PREÇOS
# AVISO DE ADIAMENTO
# AVISO DE ADJUDICAÇÃO
# AVISO DE ADJUDICAÇÃO E HOMOLOGAÇÃO
# AVISO DE ALTERAÇÃO
# AVISO DE ANULAÇÃO
# AVISO DE CHAMADA PÚBLICA
# AVISO DE CHAMAMENTO PÚBLICO
# AVISO DE HOMOLOGAÇÃO
# AVISO DE HOMOLOGAÇÃO E ADJUDICAÇÃO
# AVISO DE LICITAÇÃO
# AVISO DE LICITAÇÃO
# AVISO DE REGISTRO DE PREÇOS
# AVISO DE RETIFICAÇÃO
# AVISOS DE LICITAÇÃO
# CHAMADA PÚBLICA
# CONCORRÊNCIA
# EDITAL DE CONVOCAÇÃO
# EDITAL DE INTIMAÇÃO
# EXTRATO DE ACORDO DE COOPERAÇÃO TÉCNICA
# EXTRATO DE ADESÃO
# EXTRATO DE COMPROMISSO
# EXTRATO DE CONTRATO
# EXTRATO DE CONVÊNIO
# EXTRATO DE COOPERAÇÃO TÉCNICA
# EXTRATO DE DISPENSA DE LICITAÇÃO
# EXTRATO DE INEXIGIBILIDADE DE LICITAÇÃO
# EXTRATO DE INSTRUMENTO CONTRATUAL
# EXTRATO DE PARCERIA
# EXTRATO DE PRORROGAÇÃO DE OFÍCIO
# EXTRATO DE PROTOCOLO DE INTENÇÃO
# EXTRATO DE REGISTRO DE PREÇOS
# EXTRATO DE TERMO ADITIVO
# EXTRATO DO 2º TERMO ADITIVO
# EXTRATOS DE CONTRATOS
# EXTRATOS DE CONTRATOS
# EXTRATOS DE DOAÇÃO
# EXTRATOS DE ORDEM DE SERVIÇO
# EXTRATOS DE RESCISÃO
# EXTRATOS DE TERMOS ADITIVOS
# Extrato de Termo de Doação
# INEXIGIBILIDADE
# PREGÃO
# PREGÃO ELETRÔNICO
# PREGÃO PRESENCIAL
# PREGÃO SISPP
# PROCESSO ADMINISTRATIVO DISCIPLINAR
# PROCESSO LICITATÓRIO
# RESULTADO DA CHAMADA PÚBLICA
# RESULTADO DE HABILITAÇÃO
# RESULTADO DE JULGAMENTO
# RETIFICAÇÃO
# SISPP
# TA
# TERMO ADITIVO
# TOMADA DE PREÇOS

# Ata\sde\sRegistro\sde\sPreços